In [1]:

# https://github.com/timestocome


# Lovecraft Corpus
# https://github.com/vilmibm/lovecraftcorpus


# build word embeddings for corpus
# ~ half the words are one off and accuracy ~ 52% so that's about as well as it'll do with out
# stemming words and otherwise trimming vocabulary


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

In [3]:
# silence is golden

import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)


In [4]:

# hack to make keras work with 2*** series gpus

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [5]:
from keras.models import Sequential
from keras.layers import Conv1D, GlobalAveragePooling1D
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.preprocessing import sequence



Using TensorFlow backend.


In [6]:
# list all files under the input directory
import os

fNames = []
for dirname, _, filenames in os.walk('lovecraftcorpus'):
    for filename in filenames:
        fNames.append(os.path.join(dirname, filename))

#print(fNames)
#print(len(fNames))

In [7]:
# read in all files, split into sentences, do a bit of cleanup to reduce vocabulary size

# keep cleanup minimal 
#  convert to lower
#  convert all numbers to 9
#  remove ",'

from nltk.tokenize import sent_tokenize
import functools
import re

story_sentences = []
targets = []

for i in range(len(fNames)):
    f = fNames[i]
    fp = open(f)
    story = fp.read()
    
    # minor cleanup
    story = story.lower()
    story = re.sub('-', ' ', story)
    story = re.sub(" \'", ' ', story)
    story = re.sub('\"', ' ', story)
    story = re.sub('\d', '9', story)
    
    # break into sentences and append to the story_sentences array
    story_sentences.append(sent_tokenize(story))
    
    

 

In [8]:
# split sentences into words and get story number for each sentence to use as the target
import nltk
from nltk.tokenize import word_tokenize


story_words = []
targets = []

for i in range(len(fNames)):
    
    sentences = story_sentences[i]
    sentence_words = [word_tokenize(t) for t in sentences]
    
    targets.append( [i] * len(sentences))
    story_words.append(sentence_words)

  



In [9]:
     
# break sentences into words
combined_words = [item for sublist in story_words for item in sublist]  

# flatten sentence words, reduce to unique and sort
unique_words = sorted(set(x for s in combined_words for x in s))
n_words = len(unique_words)
print('unique words', n_words)

# flatten lists
sentences = [item for sublist in story_sentences for item in sublist]  
targets = [item for sublist in targets for item in sublist]  

# max length of sentences
maxlen = max([len(x) for x in combined_words])
print('maxlen', maxlen)



unique words 23868
maxlen 344


In [10]:
# store sentences and targets in a df
# df makes it easier to shuffle samples and pull out train/test data 

train = pd.DataFrame(targets)
train.columns = ['target']
train['words'] = combined_words

print(train.tail())





       target                                              words
18673      66  [their, deeds, i, recall, not, ,, for, they, w...
18674      66  [their, aspect, i, recall, dimly, ,, it, was, ...
18675      66  [their, name, i, recall, clearly, ,, for, it, ...
18676      66  [these, beings, of, yesterday, were, called, m...
18677      66  [so, the, genie, flew, back, to, the, thin, ho...


In [11]:
# there are faster ways to do this but this is easily reversable, works for small dataset

# convert words to int ids
def convert_word(word_list):
    
    wl = [0] * maxlen
    n = min(maxlen, len(word_list))
    
    for i in range(n):
            wl[i] = unique_words.index(word_list[i])
            
    return wl


# util to convert ints back into words  ** not used here, coded for use in future code using this as base
def convert_index(word_list):
    l = []
    for w in word_list:
        l.append(unique_words[w])
        
    return l


train['indexes'] = train['words'].apply(convert_word)
print(train.head(20))



    target                                              words  \
0        0  [beyond, the, wall, of, sleep, i, have, often,...   
1        0  [whilst, the, greater, number, of, our, noctur...   
2        0  [from, my, experience, i, can, not, doubt, but...   
3        0  [from, those, blurred, and, fragmentary, memor...   
4        0  [we, may, guess, that, in, dreams, life, ,, ma...   
5        0  [sometimes, i, believe, that, this, less, mate...   
6        0  [it, was, from, a, youthful, revery, filled, w...   
7        0  [his, name, ,, as, given, on, the, records, ,,...   
8        0  [among, these, odd, folk, ,, who, correspond, ...   
9        0  [joe, slater, ,, who, came, to, the, instituti...   
10       0  [though, well, above, the, middle, stature, ,,...   
11       0  [his, age, was, unknown, ,, since, among, his,...   
12       0  [from, the, medical, and, court, documents, we...   
13       0  [he, had, habitually, slept, at, night, beyond...   
14       0  [not, that, h

In [12]:
# convert words into oneHot vectors
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(train['target'].values.reshape(-1,1))


# shuffle data
train = train.sample(frac=1.)


# pull out 10% as validation data
n_test = len(train) // 10
n_train = len(train) - n_test
print(n_test, n_train)


# split data into train/validate and
# reshape pandas series into arrays for network
# train
x = train[0:n_train]['indexes']
x = np.asarray([y for z in x for y in z])
x = x.reshape(n_train, maxlen)

y = train[0:n_train]['target'].values.reshape(-1,1)
y = enc.transform(y)


# test
x_test = train[n_train:-1]['indexes']

x_test = np.asarray([y for z in x_test for y in z])
x_test = x_test.reshape(n_test-1, maxlen)


y_test = train[n_train:-1]['target'].values.reshape(-1,1)
y_test = enc.transform(y_test)

print(y.shape, y_test.shape)

1867 16811
(16811, 67) (1866, 67)


In [13]:
# Embedding model

n_embedding = 48



model = Sequential()

model.add(Embedding(n_words, n_embedding, input_length=maxlen))
model.add(GlobalAveragePooling1D())

model.add(Dense(len(fNames), activation='sigmoid'))

model.summary()


model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 344, 48)           1145664   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 48)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 67)                3283      
Total params: 1,148,947
Trainable params: 1,148,947
Non-trainable params: 0
_________________________________________________________________



In [14]:

# train model 
n_epochs = 300
batch_size = 16

model.fit(x, y,
         batch_size = batch_size,
         epochs = n_epochs,
         validation_data = (x_test, y_test))



Train on 16811 samples, validate on 1866 samples
Epoch 1/300
16811/16811 [==============================] - 3s 155us/step - loss: 0.0349 - acc: 0.0814 - val_loss: 0.0145 - val_acc: 0.0965
Epoch 2/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0144 - acc: 0.0929 - val_loss: 0.0143 - val_acc: 0.0965
Epoch 3/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0143 - acc: 0.0936 - val_loss: 0.0143 - val_acc: 0.0965
Epoch 4/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0143 - acc: 0.0945 - val_loss: 0.0143 - val_acc: 0.0965
Epoch 5/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0143 - acc: 0.0938 - val_loss: 0.0143 - val_acc: 0.0965
Epoch 6/300
16811/16811 [==============================] - 2s 118us/step - loss: 0.0143 - acc: 0.0938 - val_loss: 0.0143 - val_acc: 0.0965
Epoch 7/300
16811/16811 [==============================] - 2s 116us/step - loss: 0.0143 - acc: 0.0936 - val_loss: 0

16811/16811 [==============================] - 2s 115us/step - loss: 0.0114 - acc: 0.3491 - val_loss: 0.0121 - val_acc: 0.3178
Epoch 57/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0113 - acc: 0.3552 - val_loss: 0.0120 - val_acc: 0.3140
Epoch 58/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0113 - acc: 0.3560 - val_loss: 0.0120 - val_acc: 0.3264
Epoch 59/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0112 - acc: 0.3625 - val_loss: 0.0120 - val_acc: 0.3242
Epoch 60/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0111 - acc: 0.3649 - val_loss: 0.0119 - val_acc: 0.3290
Epoch 61/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0111 - acc: 0.3720 - val_loss: 0.0119 - val_acc: 0.3323
Epoch 62/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0110 - acc: 0.3781 - val_loss: 0.0119 - val_acc: 0.3280
Epoch 63/300
16811/16811 [=======

16811/16811 [==============================] - 2s 115us/step - loss: 0.0074 - acc: 0.5912 - val_loss: 0.0101 - val_acc: 0.4609
Epoch 115/300
16811/16811 [==============================] - 2s 118us/step - loss: 0.0074 - acc: 0.5909 - val_loss: 0.0101 - val_acc: 0.4625
Epoch 116/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0073 - acc: 0.5949 - val_loss: 0.0101 - val_acc: 0.4566
Epoch 117/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0073 - acc: 0.5974 - val_loss: 0.0101 - val_acc: 0.4673
Epoch 118/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0072 - acc: 0.5997 - val_loss: 0.0101 - val_acc: 0.4662
Epoch 119/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0072 - acc: 0.6026 - val_loss: 0.0101 - val_acc: 0.4609
Epoch 120/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0071 - acc: 0.6048 - val_loss: 0.0101 - val_acc: 0.4598
Epoch 121/300
16811/16811 [

16811/16811 [==============================] - 2s 116us/step - loss: 0.0051 - acc: 0.7193 - val_loss: 0.0097 - val_acc: 0.5000
Epoch 173/300
16811/16811 [==============================] - 2s 116us/step - loss: 0.0050 - acc: 0.7204 - val_loss: 0.0097 - val_acc: 0.4989
Epoch 174/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0050 - acc: 0.7219 - val_loss: 0.0096 - val_acc: 0.4989
Epoch 175/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0050 - acc: 0.7242 - val_loss: 0.0096 - val_acc: 0.4984
Epoch 176/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0049 - acc: 0.7257 - val_loss: 0.0096 - val_acc: 0.4995
Epoch 177/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0049 - acc: 0.7282 - val_loss: 0.0097 - val_acc: 0.5016
Epoch 178/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0049 - acc: 0.7284 - val_loss: 0.0096 - val_acc: 0.5005
Epoch 179/300
16811/16811 [

16811/16811 [==============================] - 2s 115us/step - loss: 0.0036 - acc: 0.8079 - val_loss: 0.0097 - val_acc: 0.5145
Epoch 231/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0036 - acc: 0.8089 - val_loss: 0.0097 - val_acc: 0.5059
Epoch 232/300
16811/16811 [==============================] - 2s 116us/step - loss: 0.0036 - acc: 0.8105 - val_loss: 0.0097 - val_acc: 0.5113
Epoch 233/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0035 - acc: 0.8120 - val_loss: 0.0097 - val_acc: 0.5080
Epoch 234/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0035 - acc: 0.8130 - val_loss: 0.0097 - val_acc: 0.5075
Epoch 235/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0035 - acc: 0.8155 - val_loss: 0.0097 - val_acc: 0.5129
Epoch 236/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0035 - acc: 0.8169 - val_loss: 0.0097 - val_acc: 0.5129
Epoch 237/300
16811/16811 [

16811/16811 [==============================] - 2s 119us/step - loss: 0.0025 - acc: 0.8777 - val_loss: 0.0097 - val_acc: 0.5188
Epoch 289/300
16811/16811 [==============================] - 2s 117us/step - loss: 0.0024 - acc: 0.8785 - val_loss: 0.0097 - val_acc: 0.5161
Epoch 290/300
16811/16811 [==============================] - 2s 116us/step - loss: 0.0024 - acc: 0.8806 - val_loss: 0.0097 - val_acc: 0.5171
Epoch 291/300
16811/16811 [==============================] - 2s 116us/step - loss: 0.0024 - acc: 0.8805 - val_loss: 0.0097 - val_acc: 0.5150
Epoch 292/300
16811/16811 [==============================] - 2s 114us/step - loss: 0.0024 - acc: 0.8816 - val_loss: 0.0097 - val_acc: 0.5102
Epoch 293/300
16811/16811 [==============================] - 2s 114us/step - loss: 0.0024 - acc: 0.8834 - val_loss: 0.0098 - val_acc: 0.5182
Epoch 294/300
16811/16811 [==============================] - 2s 115us/step - loss: 0.0024 - acc: 0.8842 - val_loss: 0.0097 - val_acc: 0.5171
Epoch 295/300
16811/16811 [

In [15]:
# retrieve embedding weights
embeddings = model.layers[0]
weights = embeddings.get_weights()[0]


# n_docs, n_words, n_embedding
weights = weights.reshape(-1, n_words, n_embedding)

print(weights.shape)

(1, 23868, 48)


In [16]:
import io


# write to file


out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(unique_words):
    vec = weights[0, num, :] 
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    
out_v.close()
out_m.close()


